# Pearson Loss Probe on Vanilla GRU
**Goal**: Test correlation-aligned losses on vanilla GRU (never tested before).
Previous Pearson loss tests were on the old complex pipeline (which had -0.034 gap).
Vanilla GRU has +0.018 gap, so loss changes may work completely differently.

**Variants**:
- **pearson_blend**: 70% CombinedLoss + 30% Pearson. Stable, moderate alignment.
- **pearson_primary**: Ramps from 30% to 60% Pearson + Huber stabilizer. Aggressive.

**Plan**: 3 seeds each. Compare to GRU base mean (0.2689).
Pass = within -0.005 of base AND produces different predictions (diversity).

In [ ]:
# Cell 0: Mount Drive, download data from Kaggle
import os, json

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

In [ ]:
# Cell 1: Setup -- clone repo, link data
import os, subprocess
REPO = "/content/competition_package"

os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"
commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Train Pearson loss variants (3 seeds each)
import os, subprocess, sys
os.chdir("/content/competition_package")

CONFIGS = [
    ("configs/vanilla_pearson_blend.yaml", "pearson_blend"),
    ("configs/vanilla_pearson_primary.yaml", "pearson_primary"),
]
SEEDS = [42, 43, 44]

print(f"=== PEARSON LOSS PROBE (Vanilla GRU) ===")
print(f"Variants: {[name for _, name in CONFIGS]}")
print(f"Seeds: {SEEDS}")
print("=" * 60, flush=True)

for config_path, variant_name in CONFIGS:
    for seed in SEEDS:
        print(f"\n{'='*60}")
        print(f"Training {variant_name} seed {seed}")
        print(f"{'='*60}", flush=True)
        proc = subprocess.Popen(
            [sys.executable, "-u", "scripts/train.py",
             "--config", config_path,
             "--seed", str(seed), "--device", "cuda"],
            stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
        )
        for line in proc.stdout:
            print(line, end="", flush=True)
        proc.wait()
        if proc.returncode != 0:
            print(f"ERROR: {variant_name} seed {seed} failed with rc={proc.returncode}")

print(f"\nAll training done!")

In [ ]:
# Cell 3: Evaluate all Pearson loss models
import os, glob, torch
os.chdir("/content/competition_package")

GRU_BASE_MEAN = 0.2689  # mean of 23 GRU base seeds (MSE loss)

for variant in ["vanilla_pearson_blend", "vanilla_pearson_primary"]:
    results = []
    for pt in sorted(glob.glob(f"logs/{variant}_seed*.pt")):
        basename = os.path.basename(pt)
        if '_epoch' in basename:
            continue
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
        score = float(ckpt.get("best_score", 0))
        epoch = ckpt.get("best_epoch", "N/A")
        seed = int(basename.split("seed")[1].split(".")[0])
        results.append((seed, score, epoch))
    
    results.sort(key=lambda x: -x[1])
    
    print(f"\n=== {variant} ===")
    print(f"{'Rank':<6} {'Seed':<6} {'Val Score':>10} {'Epoch':>6}")
    print("-" * 35)
    for i, (seed, score, epoch) in enumerate(results, 1):
        print(f"{i:<6} s{seed:<5} {score:>10.4f} {str(epoch):>6}")
    
    if results:
        scores = [s for _, s, _ in results]
        mean_val = sum(scores) / len(scores)
        delta = mean_val - GRU_BASE_MEAN
        print(f"\n{variant}: {len(results)} seeds, mean={mean_val:.4f}, delta={delta:+.4f} vs MSE base")
        if delta >= -0.005:
            print(">>> PASS: Quality acceptable <<<")
        else:
            print(">>> FAIL: Quality too low, KILL <<<")

In [ ]:
# Cell 4: Strip checkpoints + zip + save to Drive
import os, torch, glob, shutil
os.chdir("/content/competition_package")
os.makedirs("logs/slim", exist_ok=True)

count = 0
for variant in ["vanilla_pearson_blend", "vanilla_pearson_primary"]:
    for pt in sorted(glob.glob(f"logs/{variant}_seed*.pt")):
        basename = os.path.basename(pt)
        if '_epoch' in basename:
            continue
        ckpt = torch.load(pt, map_location="cpu", weights_only=False)
        slim = {
            "model_state_dict": ckpt["model_state_dict"],
            "config": ckpt.get("config", {}),
            "best_score": ckpt.get("best_score", None),
            "best_epoch": ckpt.get("best_epoch", None),
        }
        out = f"logs/slim/{basename}"
        torch.save(slim, out)
        orig = os.path.getsize(pt) / 1e6
        new = os.path.getsize(out) / 1e6
        print(f"{basename}: {orig:.1f}MB -> {new:.1f}MB")
        count += 1

print(f"\nStripped {count} checkpoints")

shutil.make_archive("/content/pearson_loss_seeds", "zip",
                     "/content/competition_package/logs/slim")
sz = os.path.getsize("/content/pearson_loss_seeds.zip") / 1e6
print(f"\npearson_loss_seeds.zip: {sz:.1f}MB")

shutil.copy("/content/pearson_loss_seeds.zip",
            "/content/drive/MyDrive/wunderfund/pearson_loss_seeds.zip")
print("Saved to Drive: MyDrive/wunderfund/pearson_loss_seeds.zip")